In [27]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import warnings

pd.options.mode.chained_assignment = None
warnings.simplefilter(action='ignore', category=FutureWarning)

In [28]:
url_to_scrape = [
    'https://www.dreamwaretech.co.za/c/computer-components/graphics-cards-gpus/nvidia-graphics-cards/',
    'https://www.dreamwaretech.co.za/c/computer-components/graphics-cards-gpus/workstation-cards/',
    'https://www.dreamwaretech.co.za/c/computer-components/graphics-cards-gpus/amd-graphics-cards/'
]

df = pd.DataFrame(columns=["Title", "Price", "In Stock", "Category"])

In [29]:
for base_url in url_to_scrape:
    print(base_url)

    paging = True
    url = base_url + ""
    while paging:
        html_content = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
        soup = BeautifulSoup(html_content.text)
        divs = soup.find_all("div", class_= "product card")

        for product in divs:
            price = float(product.find("p", class_="product-price").text.split(' ')[1].replace('R', ''))
            title = product.find("p", class_="product-box-name").find("a").get("href").split("/")[-2].replace("-", " ")
            in_stock = True if "with supplier" in product.find("p", class_="prod-availability").text.lower() else False
        
            df = pd.concat([df, pd.DataFrame({"Title": title, "Price": price, "In Stock": in_stock, "Category": "Graphics Cards"}, index=[0])], ignore_index=True)
        
        if soup.find("p", id="next-nav"):
            paging = True if soup.find("p", id="next-nav").text in "Last" else False
            next_page_url = soup.find("a", id="a-next-nav").get("href")

            if next_page_url:
                url = base_url + next_page_url
            else:
                paging = False
        else:
            paging = False


df.shape

https://www.dreamwaretech.co.za/c/computer-components/graphics-cards-gpus/nvidia-graphics-cards/
https://www.dreamwaretech.co.za/c/computer-components/graphics-cards-gpus/workstation-cards/
https://www.dreamwaretech.co.za/c/computer-components/graphics-cards-gpus/amd-graphics-cards/


(143, 4)

In [30]:
df.to_csv("DreamWareTech.csv", index=False)

In [47]:
def word_similarity(str1, str2):
    str1 = str1.lower().split(" ")
    str2 = str2.lower().split(" ")
    count = 0 
    for word in str1:
        if word in str2:
            count += 1
    return count 

    
    

#Compute the jaccaard similarity between the test card and all the cards in the dataset
df["Title"].apply(lambda x: word_similarity(x, "gigabyte geforce rtx 4060 ti aero")).sort_values(ascending=False).head(10)



4     6
15    6
0     5
13    5
17    5
16    5
14    5
28    5
2     5
75    4
Name: Title, dtype: int64

In [48]:
#Print the row at 62
df.iloc[4]

Title       gigabyte geforce rtx 4060 ti aero oc 16gb gddr...
Price                                                 13169.0
In Stock                                                 True
Category                                       Graphics Cards
Name: 4, dtype: object

2